In [ ]:
!pip install telegram-send

In [ ]:
import requests
def telegram_bot_sendtext(bot_message):
    
    bot_token = ''
    bot_chatID = ''
    send_text = 'https://api.telegram.org/bot' + bot_token + '/sendMessage?chat_id=' + bot_chatID + '&parse_mode=Markdown&text=' + bot_message

    response = requests.get(send_text)

    return response.json()

In [ ]:
import time

In [ ]:
import ibmos2spark, os
# @hidden_cell

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_6337e1afa96b41479fda20829220b743 = 'https://s3.us.cloud-object-storage.appdomain.cloud'
else:
    endpoint_6337e1afa96b41479fda20829220b743 = 'https://s3.private.us.cloud-object-storage.appdomain.cloud'

credentials = {
    'endpoint': endpoint_6337e1afa96b41479fda20829220b743,
    'service_id': '',
    'iam_service_endpoint': 'https://iam.cloud.ibm.com/oidc/token',
    'api_key': '',
    'access_key': '',
    'secret_key': ''
}

configuration_name = 'spark_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

In [ ]:
from pyspark.sql.types import *

schema = StructType() \
          .add("qid", IntegerType(), True) \
          .add("question", StringType(), True) \
          .add("vots", IntegerType(), True) \
          .add("answers", IntegerType(), True) \
          .add("views",  IntegerType(), True) \
          .add("tags", StringType(), True) \
          .add("created_at", TimestampType(), True) \
          .add("last_modifyed", TimestampType(), True)

In [ ]:
from pyspark.sql.functions import udf
import re
def strArray_to_str(my_str):
    my_str = my_str[1:-1]
    my_str = re.sub(',',  '',    my_str)
    my_str = re.sub("'",  '', my_str)   
    #return my_str.split(" ")
    return ','.join(my_str.split(" "))
str_to_arr_udf = udf(strArray_to_str,StringType())

### save in a database

In [ ]:
# @hidden_cell

# This connection object is used to access your data and contains your credentials or project token.
# You might want to remove those credentials before you share your notebook.

import os
from pyspark.sql import SparkSession
sparkSession = SparkSession(spark).builder.getOrCreate()

db2_connection_url = 'jdbc:db2://{}:{}/{}:sslConnection=true;'.format(
    f'{}.databases.appdomain.cloud',
    32733,
    'bludb'
)

In [ ]:
user = ''
password = """"""
prop = {"user":user, "password":password}

### monitor 

## S

In [ ]:
def executeDF(df, epochId):
    #df = df.withColumn("tags",str_to_arr_udf(df["tags"]))
    df.write.jdbc(url=db2_connection_url , table='"BCZ79269"."SO_QUESTIONS"', mode="overwrite",properties=prop)
    message = "🚀 Scraping\n\nO observador está de pé"
    telegram_bot_sendtext(message)    

In [ ]:
spark.conf.set("spark.sql.shuffle.partitions", "1")

In [ ]:
# apenas le os arquivos no bucket para dentro de um dataframe
bucket = 'sparkbucketwatcheripynb-donotdelete-pr-3je2emjupdkz5l'
obj = '*.csv'

In [ ]:
query = (
  sparkSession.readStream.format("csv").schema(schema).option('sep', ';').load(cos.url(obj, bucket))
    .writeStream
    .foreachBatch(executeDF)
    .outputMode("update")
    .start()
    )